# 1. 環境安裝

* 記得要先將HuggingFace Api KEY設定於Secrets"HF_TOKEN"
* 買不起5090的免費仔只好用colab練模型

In [ ]:
# 1. 安裝環境
!pip install -q datasets transformers torch accelerate setfit scikit-learn matplotlib seaborn
!pip install hf_xet
!pip install openpyxl

# 2. 掛載 Google Drive
from google.colab import drive
import os
import shutil

drive.mount('/content/drive')

# 設定你的檔案在 Drive 中的路徑
DRIVE_PATH = "/content/drive/MyDrive/Colab Notebooks/Project/Sarcasm-Enhanced-SetFit/imdb_analysis_results"
os.makedirs(DRIVE_PATH, exist_ok=True) # 自動建立資料夾

# 檢查一下檔案在不在
if os.path.exists(DRIVE_PATH):
    print(f"✅ 成功連接 Drive，檔案列表: {os.listdir(DRIVE_PATH)}")
else:
    print("❌ 找不到路徑，請確認你的資料夾名稱是否正確")


DRIVE_SAVE_PATH = "/content/drive/MyDrive/Colab Notebooks/Project/Sarcasm-Enhanced-SetFit/Model_colab"
os.makedirs(DRIVE_SAVE_PATH, exist_ok=True) # 自動建立資料夾

print(f"📂 模型將儲存於雲端路徑: {DRIVE_SAVE_PATH}")

In [ ]:
# 各項基本設定
USER_CONFIG = {
    "data": {
        "dataset_name": "imdb",
        "split": "train",
        "seed_count": 800,  # 練習用種子資料數量
        "random_seed": 42
    },
    "llm": {
        "model_id": "Qwen/Qwen2.5-1.5B-Instruct",
        "temperature": 0.1,  # 降低隨機性，保證標註一致性
        "max_new_tokens": 100
    },
    "labels": {
        "aspects": ["Acting", "Plot/Story", "Visuals/Effects", "Pacing"],
        "sarcasm": ["Yes", "No"]
    }
}

# 2. 核心程式碼：GenAI標註

* 這個部分是利用下載HF上的開源GenAI模型(這邊選的是 **Qwen/Qwen2.5-1.5B-Instruct** )將初始資料快速做分類以及加上反諷標註，作為後續訓練資料。

## 標註資料主程式

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd
from tqdm import tqdm
import json
import re

# ==========================================
# 1. 模組化函式 (Modules)
# ==========================================

def load_raw_data(config):
    """載入 IMDB 資料集並取樣"""
    print(f"Loading {config['data']['dataset_name']} dataset...")
    dataset = load_dataset(config['data']['dataset_name'], split=config['data']['split'])

    # 隨機打亂並取前 N 筆
    sampled_data = dataset.shuffle(seed=config['data']['random_seed']).select(range(config['data']['seed_count']))
    print(f"Data loaded. Sample size: {len(sampled_data)}")
    return sampled_data

def setup_genai_model(config):
    """初始化 Qwen 模型與 Pipeline"""
    print(f"Loading LLM: {config['llm']['model_id']}...")

    tokenizer = AutoTokenizer.from_pretrained(config['llm']['model_id'])
    model = AutoModelForCausalLM.from_pretrained(
        config['llm']['model_id'],
        torch_dtype=torch.float16,
        device_map="auto"
    )

    # 建立 Text Generation Pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=config['llm']['max_new_tokens'],
        temperature=config['llm']['temperature'],
        do_sample=True # Set to True for temperature to take effect, but keep temp low
    )
    return pipe

def construct_prompt(review_text):
    """
    建構 Prompt 工程：
    要求模型扮演影評分析師，並輸出嚴格的 JSON 格式。
    """

    system_prompt = f"""You are a strict film critic AI.
Your task is to classify the user's review into EXACTLY ONE category: [Acting, Visuals/Effects, Pacing, Plot/Story].

**CRITICAL INSTRUCTION**: You must follow this priority order to decide:

1. **PRIORITY 1 - Acting**: Does the review mention specific actor names, performance quality (stiff, amazing), or character believability?
   -> If YES, label as "Acting". (STOP HERE)

2. **PRIORITY 2 - Visuals/Effects**: Does the review discuss CGI, lighting, camera work, atmosphere, or cinematography?
   -> If YES, label as "Visuals/Effects". (STOP HERE)

3. **PRIORITY 3 - Pacing**: Does the review explicit complain about the movie's speed (too slow/fast) or length?
   -> If YES, label as "Pacing". (STOP HERE)

4. **FALLBACK - Plot/Story**: Only if NONE of the above apply, and the review discusses the script, narrative, or general feelings.
   -> Label as "Plot/Story".

Output strictly in JSON format:
{{"aspect": "Acting", "sarcasm": "No"}}
"""

    user_prompt = f"Review: \"{review_text[:1000]}\"..." # 截斷過長評論以節省資源

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    return messages

def parse_llm_output(output_text):
    """解析 LLM 輸出的 JSON"""
    try:
        # 嘗試用 Regex 抓取 JSON區塊，避免模型輸出額外廢話
        match = re.search(r'\{.*\}', output_text, re.DOTALL)
        if match:
            json_str = match.group(0)
            data = json.loads(json_str)
            return data.get("aspect", "Unknown"), data.get("sarcasm", "No")
    except:
        pass
    return "Unknown", "No" # Fallback

def run_auto_labeling(dataset, llm_pipe):
    """執行批次標註"""
    results = []
    print("Starting GenAI Auto-Annotation...")

    for item in tqdm(dataset):
        messages = construct_prompt(item['text'])

        # 套用 Chat Template
        prompt = llm_pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        # 生成
        outputs = llm_pipe(prompt)
        generated_text = outputs[0]['generated_text']

        # 提取模型回答的部分 (去掉 Prompt)
        response = generated_text.split("<|im_start|>assistant\n")[-1].strip()

        # 解析
        aspect, sarcasm = parse_llm_output(response)

        results.append({
            "text": item['text'],
            "label_original": item['label'], # IMDB 原本的 sentiment (0/1)
            "aspect": aspect,
            "sarcasm": sarcasm
        })

    return pd.DataFrame(results)

# ==========================================
# 2. 主執行區塊 (Main Execution)
# ==========================================

# A. 載入資料
sampled_dataset = load_raw_data(USER_CONFIG)

# B. 設定模型
llm_pipeline = setup_genai_model(USER_CONFIG)

# C. 執行自動標註
labeled_df = run_auto_labeling(sampled_dataset, llm_pipeline)

# D. 檢查結果
print("\n=== Annotation Sample ===")
print(labeled_df[['aspect', 'sarcasm']].value_counts())
display(labeled_df.head(5))

## 儲存標註完成資料

In [ ]:
# 1. 確認你的輸出資料夾存在
OUTPUT_DIR = DRIVE_PATH
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 2. 標註資料在 'labeled_df' 這個變數裡
if 'labeled_df' in locals():
    print(f"找到標註資料，共 {len(labeled_df)} 筆。正在存檔...")

    # --- 儲存為 CSV (推薦給之後訓練模型讀取) ---
    csv_path = os.path.join(OUTPUT_DIR, "qwen_labeled_data.csv")
    labeled_df.to_csv(csv_path, index=False, encoding='utf-8-sig') # sig 可以避免 Excel 打開亂碼
    print(f"✅ CSV 已儲存: {os.path.abspath(csv_path)}")

    # --- 儲存為 Excel (推薦用來人工檢查/修正標籤) ---
    excel_path = os.path.join(OUTPUT_DIR, "qwen_labeled_data.xlsx")
    labeled_df.to_excel(excel_path, index=False)
    print(f"✅ Excel 已儲存: {os.path.abspath(excel_path)}")

else:
    print("❌ 錯誤：在記憶體中找不到 'labeled_df' 這個變數！")
    print("請確認你已經跑完 QWEN 的標註區塊，或者檢查變數名稱是否不同。")

## 檢查標註資料

In [ ]:
def check_per_category(df, samples_per_class=3):
    categories = df['aspect'].unique()

    for cat in categories:
        print(f"\n🔵 Checking Category: [{cat}]")
        print("="*60)

        # 該分類中隨機抽 N 筆，如果不夠 N 筆就全抓
        subset = df[df['aspect'] == cat]
        n = min(len(subset), samples_per_class)

        if n == 0:
            print("(No data found for this category)")
            continue

        sample_rows = subset.sample(n)

        for idx, row in sample_rows.iterrows():
            print(f"🔴 Sarcasm: {row['sarcasm']}")
            print(f"📝 Review: {row['text'][:400]}...") # 顯示前 400 字
            print("-" * 30)

# 執行檢查
check_per_category(labeled_df)

# 3. SetFit評論類型分類模型訓練

In [ ]:
import gc
import torch

print("Cleaning up GPU memory...")

# 1. 刪除 LLM Pipeline 物件 (切斷對模型的引用)
try:
    del llm_pipeline
except NameError:
    pass

# 2. 強制執行 Python 垃圾回收
gc.collect()

# 3. 清空 PyTorch 的 GPU 快取 (這是最關鍵的一步)
torch.cuda.empty_cache()

print("GPU Memory Cleared! Ready for training.")

# 檢查現在剩餘多少記憶體 (選用)
# print(torch.cuda.memory_summary(abbreviated=True))

## A. 未加Sarcasm標記

In [ ]:
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sentence_transformers.losses import CosineSimilarityLoss

# ==========================================
# 未加Sarcasm標記 : SetFit 模型訓練
# ==========================================

def prepare_dataset_for_setfit(df):
    """
    資料前處理：
    SetFit 需要 standard 的 HuggingFace Dataset 格式。
    """
    # 1. 將文字標籤轉為數字 ID (Label Encoding)
    aspect_map = {label: i for i, label in enumerate(USER_CONFIG['labels']['aspects'])}
    # 注意：如果有模型標註出 "Unknown" 或其他怪東西，我們要過濾掉或是歸類
    df_clean = df[df['aspect'].isin(USER_CONFIG['labels']['aspects'])].copy()

    df_clean['label'] = df_clean['aspect'].map(aspect_map)

    print(f"Valid data for training: {len(df_clean)} / {len(df)}")
    print(f"Label Mapping: {aspect_map}")

    # 2. 切分訓練集與驗證集
    train_df, eval_df = train_test_split(df_clean, test_size=0.2, random_state=42)

    # 3. 轉為 HF Dataset
    train_ds = Dataset.from_pandas(train_df)
    eval_ds = Dataset.from_pandas(eval_df)

    return train_ds, eval_ds, aspect_map


def train_efficient_setfit(full_train_dataset, eval_dataset, label_mapping):
    """
    資料中四個種類分布不平均，以Pacing最少、Plot最多又最雜。
    根據這個原則設定不同數量丟入SetFit訓練
    """
    # 1. 轉回 Pandas 做精準抽樣
    df_train = full_train_dataset.to_pandas()

    SAMPLE_QUOTAS = {
        0: 100,  # Acting: 給 100 筆
        1: 180,  # Plot: 給 180 筆
        2: 50,   # Visuals: 50
        3: 50    # Pacing: 50
    }

    def dynamic_sample(group):
        label_id = group.name # 取得目前的 label ID
        # 取得該類別的配額，如果沒設定就預設 50
        quota = SAMPLE_QUOTAS.get(label_id, 50)
        # 實際抽樣：取 (配額) 與 (該類別總數) 的最小值
        return group.sample(min(len(group), quota), random_state=42)


    # 執行抽樣
    df_sampled = df_train.groupby('label', group_keys=False).apply(dynamic_sample)

    # 強制重置 Index
    df_sampled = df_sampled.reset_index(drop=True)

    # 轉回 Dataset
    train_subset = Dataset.from_pandas(df_sampled, preserve_index=False)

    print(f"原本訓練集: {len(full_train_dataset)} -> 📉 平衡抽樣後: {len(train_subset)}")
    print(f"抽樣後的類別分布:\n{df_sampled['label'].value_counts()}")

    # 2. 載入模型
    model_id = "sentence-transformers/paraphrase-mpnet-base-v2"
    model = SetFitModel.from_pretrained(
        model_id,
        id2label={v: k for k, v in label_mapping.items()},
        label2id=label_mapping,
    )

    # 3. 設定訓練參數
    trainer = SetFitTrainer(
        model=model,
        train_dataset=train_subset, # 餵入抽樣後的資料
        eval_dataset=eval_dataset,
        loss_class=CosineSimilarityLoss,
        metric="accuracy",

        # --- 效率參數區 ---
        batch_size=8,
        num_iterations=12,
        num_epochs=1,
        column_mapping={"text": "text", "label": "label"}
    )

    # 4. 開始訓練
    print("🚀 開始高效率訓練 (Efficient Training)...")
    trainer.train()

    # 5. 評估
    metrics = trainer.evaluate()
    print(f"\n📊 訓練完成。Metrics: {metrics}")

    return model

# ==========================================
# 執行區塊
# ==========================================

if 'labeled_df' in locals():

    print("Found labeled_df, preparing datasets...")
    # 1. 準備資料
    train_dataset, eval_dataset, label_mapping = prepare_dataset_for_setfit(labeled_df)

    # 2. 訓練模型
    model_a = train_efficient_setfit(train_dataset, eval_dataset, label_mapping)

    # 3. 保存模型
    # 雲端
    save_path_a = os.path.join(DRIVE_SAVE_PATH, "IMDB_aspect_model")
    model_a.save_pretrained(save_path_a)
    print(f"✅ 模型本體已儲存: {save_path_a}")

else:
    raise ValueError("找不到 labeled_df！")

## B. 加上Sarcasm標記

In [ ]:
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sentence_transformers.losses import CosineSimilarityLoss

# ==========================================
# 加上Sarcasm標記 : SetFit 模型訓練
# ==========================================
def prepare_dataset_for_model_b(df):
    """
    Model B 專用：資料前處理
    差異點：將 Sarcasm 資訊注入到 Text 中 (Text Augmentation)
    """

    # 1. 基礎過濾
    aspect_map = {label: i for i, label in enumerate(USER_CONFIG['labels']['aspects'])}
    df_clean = df[df['aspect'].isin(USER_CONFIG['labels']['aspects'])].copy()

    print("正在加入反諷標記...")

    def inject_tag(row):
        # 如果是反諷，就在句子最前面加上 [Sarcasm]
        if row['sarcasm'] == 'Yes':
            return f"[Sarcasm] {row['text']}"
        else:
            return row['text'] # 不是反諷就保持原樣

    # 覆蓋原本的 text 欄位
    df_clean['text'] = df_clean.apply(inject_tag, axis=1)

    # 檢查一下有沒有成功
    print(f"範例 (前3筆反諷資料):\n{df_clean[df_clean['sarcasm']=='Yes'][['text', 'aspect']].head(3)}")
    # ==========================================

    # 3. 標籤編碼 (跟之前一樣)
    df_clean['label'] = df_clean['aspect'].map(aspect_map)

    # 4. 切分 (跟之前一樣)
    train_df, eval_df = train_test_split(df_clean, test_size=0.2, random_state=42)

    # 5. 轉為 HF Dataset
    train_ds = Dataset.from_pandas(train_df)
    eval_ds = Dataset.from_pandas(eval_df)

    return train_ds, eval_ds, aspect_map


def train_efficient_setfit(full_train_dataset, eval_dataset, label_mapping):

    # 1. 轉回 Pandas 做精準抽樣
    df_train = full_train_dataset.to_pandas()

    SAMPLE_QUOTAS = {
        0: 100,  # Acting: 給 100 筆 (原本 40 不夠)
        1: 180,  # Plot: 給 150 筆 (這是最難的，給最多)
        2: 50,   # Visuals: 50 筆就夠強了
        3: 50    # Pacing: 50 筆就夠強了
    }

    def dynamic_sample(group):
        label_id = group.name # 取得目前的 label ID
        # 取得該類別的配額，如果沒設定就預設 50
        quota = SAMPLE_QUOTAS.get(label_id, 50)
        # 實際抽樣：取 (配額) 與 (該類別總數) 的最小值
        return group.sample(min(len(group), quota), random_state=42)


    # 執行抽樣
    df_sampled = df_train.groupby('label', group_keys=False).apply(dynamic_sample)

    # 強制重置 Index
    df_sampled = df_sampled.reset_index(drop=True)

    # 轉回 Dataset
    train_subset = Dataset.from_pandas(df_sampled, preserve_index=False)

    print(f"原本訓練集: {len(full_train_dataset)} -> 📉 平衡抽樣後: {len(train_subset)}")
    print(f"抽樣後的類別分布:\n{df_sampled['label'].value_counts()}")

    # 2. 載入模型
    model_id = "sentence-transformers/paraphrase-mpnet-base-v2"
    model = SetFitModel.from_pretrained(
        model_id,
        id2label={v: k for k, v in label_mapping.items()},
        label2id=label_mapping,
    )

    # 3. 設定訓練參數
    trainer = SetFitTrainer(
        model=model,
        train_dataset=train_subset, # 餵入抽樣後的資料
        eval_dataset=eval_dataset,
        loss_class=CosineSimilarityLoss,
        metric="accuracy",

        # --- 效率參數區 ---
        batch_size=8,
        num_iterations=12,
        num_epochs=1,
        column_mapping={"text": "text", "label": "label"}
    )

    # 4. 開始訓練
    print("🚀 開始高效率訓練 (Efficient Training)...")
    trainer.train()

    # 5. 評估
    metrics = trainer.evaluate()
    print(f"\n📊 訓練完成。Metrics: {metrics}")

    return model


# ==========================================
# 執行區塊
# ==========================================

if 'labeled_df' in locals():
    print("🚀 開始訓練 Model B (Sarcasm Enhanced)...")

    # 1. 準備「加工過」的資料
    train_ds_b, eval_ds_b, label_map_b = prepare_dataset_for_model_b(labeled_df)

    # 2. 訓練模型
    model_b = train_efficient_setfit(train_ds_b, eval_ds_b, label_map_b)

    # 3. 另外存檔
    # 雲端
    save_path_b = os.path.join(DRIVE_SAVE_PATH, "IMDB_aspect_model_enhanced")
    model_b.save_pretrained(save_path_b)

    print(f"✅ Model B (反諷增強版) 已儲存至: {save_path_b}")

else:
    print("❌ 找不到 labeled_df，請先讀取資料。")

# 4. SetFit情感分類(正負評)模型訓練

In [ ]:
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset
from sentence_transformers.losses import CosineSimilarityLoss
import pandas as pd
import os
import gc
import torch

# 設定存檔路徑
OUTPUT_DIR = DRIVE_SAVE_PATH

# ==========================================
# 0. 讀取與檢查資料
# ==========================================
csv_file = os.path.join(DRIVE_PATH, "qwen_labeled_data.csv")

if os.path.exists(csv_file):
    labeled_df = pd.read_csv(csv_file)
    print(f"✅ 成功載入資料: {len(labeled_df)} 筆")
    print(f"   標籤分佈:\n{labeled_df['label_original'].value_counts()}")
else:
    raise ValueError(f"❌ 找不到 {csv_file}，請確認檔案是否已上傳Colab。")

# ==========================================
# 1. 資料準備函式：情感分析專用
# ==========================================
def prepare_sentiment_dataset(df, use_sarcasm=False):
    print(f"\n📦 正在準備情感分析資料集 (Sarcasm Mode: {use_sarcasm})...")

    # 複製資料
    df_clean = df.copy()

    # A. 設定訓練目標 (直接使用 label_original)
    df_clean['label'] = df_clean['label_original']

    # B. 注入反諷標籤 (如果是 Model B)
    if use_sarcasm:
        print("   ✨ 注入 [Sarcasm] 特徵中...")
        def inject_tag(row):
            # 注意：這裡對應 csv 中的 'Yes'
            if row['sarcasm'] == 'Yes':
                return f"[Sarcasm] {row['text']}"
            else:
                return row['text']
        df_clean['text'] = df_clean.apply(inject_tag, axis=1)

    # C. 抽樣策略
    # 資料量約 800 筆，SetFit 跑全量也非常快，這裡我們設定上限為 300 筆/類 (保留一點測試空間)
    SAMPLES_PER_CLASS = 300

    print(f"   📉 執行平衡抽樣：每類上限 {SAMPLES_PER_CLASS} 筆...")
    df_sampled = df_clean.groupby('label', group_keys=False).apply(
        lambda x: x.sample(min(len(x), SAMPLES_PER_CLASS), random_state=42)
    )
    df_sampled = df_sampled.reset_index(drop=True)

    # D. 轉為 HF Dataset
    train_ds = Dataset.from_pandas(df_sampled, preserve_index=False)

    print(f"   ✅ 準備完成！訓練集大小: {len(train_ds)}")
    return train_ds

# ==========================================
# 2. 訓練函式
# ==========================================
def train_sentiment_model(train_ds, model_name_suffix):

    model_id = "sentence-transformers/paraphrase-mpnet-base-v2"

    print(f"\n🚀 開始訓練 Sentiment Model ({model_name_suffix})...")

    # 載入模型
    model = SetFitModel.from_pretrained(model_id)

    # 訓練設定
    trainer = SetFitTrainer(
        model=model,
        train_dataset=train_ds,
        loss_class=CosineSimilarityLoss,
        metric="accuracy",
        batch_size=8,
        num_iterations=5,
        num_epochs=1,
        column_mapping={"text": "text", "label": "label"}
    )

    trainer.train()
    print(f"✅ {model_name_suffix} 訓練完成！")

    return model

# ==========================================
# 3. 執行訓練區塊
# ==========================================
gc.collect()
torch.cuda.empty_cache()

# --- 訓練 Model A (Baseline: 無反諷) ---
train_ds_a = prepare_sentiment_dataset(labeled_df, use_sarcasm=False)
sentiment_model_a = train_sentiment_model(train_ds_a, "Baseline")

# 存檔
save_path_a = os.path.join(OUTPUT_DIR, "Sentiment_Model_A_Baseline")
sentiment_model_a.save_pretrained(save_path_a)

############################################

gc.collect()
torch.cuda.empty_cache()

# --- 訓練 Model B (Enhanced: 有反諷) ---
train_ds_b = prepare_sentiment_dataset(labeled_df, use_sarcasm=True)
sentiment_model_b = train_sentiment_model(train_ds_b, "Sarcasm-Enhanced")

# 存檔
save_path_b = os.path.join(OUTPUT_DIR, "Sentiment_Model_B_Enhanced")
sentiment_model_b.save_pretrained(save_path_b)

print(f"\n🎉 兩組模型皆已訓練完成！請至 {OUTPUT_DIR} 查看。")